In [5]:
import sys  
sys.path.insert(0, '../..')

In [ ]:
%load_ext autoreload
%autoreload 2

In [7]:
import optuna
from optuna.samplers import TPESampler
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

import logging
import sys

In [13]:
def objective(trial):
    seed = 0

    rock_results = []
    other_results = []

    n_samples = trial.suggest_int('n_samples', 1, 16) * 100
    jitter = trial.suggest_int('jitter', 1, 20) * 0.01


    datasets = []
    n_centers = None
    
    for s in range(seed, seed+10):
        datasets.append(make_moons(n_samples=n_samples, noise=jitter,
            shuffle=True, random_state=s))
        
    step = 0
    for dataset in datasets: 
        run = {}
        run['run_id'] = run_id
        run['step'] = step
        run['n_samples'] = n_samples
        run['jitter'] = jitter
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])
        kmeans = KMeans(n_clusters=2, random_state=seed).fit(data).labels_
        run['k'] = 2
        eps, min_pts = (0.2, 4)
        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = min_pts
        spectral = SpectralClustering(n_clusters=2, random_state=seed).fit(data).labels_
        bandwidth = estimate_bandwidth(data)
        run['bandwidth'] = bandwidth
        mean_shift = MeanShift(bandwidth=bandwidth).fit(data).labels_

        rock = ROCK(tmax=15).fit(data).labels_

        gt = dataset[1]
        rock_results.append(AMI(rock, gt))
        other_results.append(np.max([AMI(kmeans, gt), AMI(dbscan, gt), AMI(spectral, gt), AMI(mean_shift, gt)]))

        run['ROCK'] = AMI(rock, gt)
        run['K_MEANS'] = AMI(kmeans, gt)
        run['DBSCAN'] = AMI(dbscan, gt)
        run['SPECTRAL'] = AMI(spectral, gt)
        run['MEAN_SHIFT'] = AMI(mean_shift, gt)

        step += 1

        experiment.append(run)

    return np.mean(rock_results) - np.mean(other_results)

In [14]:
experiment = []
run_id = 0

In [15]:
study_name = 'two_moons'

In [17]:
storage_name = "sqlite:///{}.db".format(study_name)
sampler = TPESampler(seed=42)
study = optuna.create_study(study_name=study_name, storage=storage_name, sampler=sampler, load_if_exists=True,  direction="maximize")

[I 2021-05-30 16:36:25,748] A new study created in RDB with name: two_moons


In [18]:
study.optimize(objective, n_trials=100)

[I 2021-05-30 16:37:07,287] Trial 0 finished with value: 0.17165110355273971 and parameters: {'n_samples': 6, 'jitter': 20}. Best is trial 0 with value: 0.17165110355273971.
[I 2021-05-30 16:38:21,949] Trial 1 finished with value: 0.1707221500535816 and parameters: {'n_samples': 12, 'jitter': 12}. Best is trial 0 with value: 0.17165110355273971.
[I 2021-05-30 16:38:35,735] Trial 2 finished with value: -0.1777327860237462 and parameters: {'n_samples': 3, 'jitter': 4}. Best is trial 0 with value: 0.17165110355273971.
[I 2021-05-30 16:38:40,703] Trial 3 finished with value: 0.100519992181483 and parameters: {'n_samples': 1, 'jitter': 18}. Best is trial 0 with value: 0.17165110355273971.
[I 2021-05-30 16:39:41,702] Trial 4 finished with value: 0.3580777415846451 and parameters: {'n_samples': 10, 'jitter': 15}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 16:39:46,674] Trial 5 finished with value: 0.09615812279639446 and parameters: {'n_samples': 1, 'jitter': 20}. Best is t

[I 2021-05-30 17:59:15,362] Trial 44 finished with value: 0.3196228543529997 and parameters: {'n_samples': 10, 'jitter': 14}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 17:59:57,003] Trial 45 finished with value: 0.3462561084025223 and parameters: {'n_samples': 9, 'jitter': 12}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 18:00:37,478] Trial 46 finished with value: 0.2418578202112659 and parameters: {'n_samples': 8, 'jitter': 15}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 18:01:42,678] Trial 47 finished with value: 0.24642902178190923 and parameters: {'n_samples': 12, 'jitter': 17}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 18:02:15,569] Trial 48 finished with value: 0.1671843432233151 and parameters: {'n_samples': 7, 'jitter': 19}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 18:03:06,570] Trial 49 finished with value: -0.24286667062275336 and parameters: {'n_samples': 11, 'jitter': 10}. Best

[I 2021-05-30 18:51:21,828] Trial 88 finished with value: 0.3580777415846451 and parameters: {'n_samples': 10, 'jitter': 15}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 18:52:25,640] Trial 89 finished with value: 0.27018874201047866 and parameters: {'n_samples': 12, 'jitter': 14}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 18:53:07,338] Trial 90 finished with value: 0.2852639926243666 and parameters: {'n_samples': 9, 'jitter': 13}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 18:53:56,813] Trial 91 finished with value: 0.3580777415846451 and parameters: {'n_samples': 10, 'jitter': 15}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 18:54:46,340] Trial 92 finished with value: 0.3580777415846451 and parameters: {'n_samples': 10, 'jitter': 15}. Best is trial 4 with value: 0.3580777415846451.
[I 2021-05-30 18:55:44,860] Trial 93 finished with value: 0.26614851507529974 and parameters: {'n_samples': 11, 'jitter': 16}. Bes

In [19]:
pd.DataFrame(experiment).to_csv('two_moons.csv')